# Email A/B Test
Half of all email adds were generic upsell ads (variant="control"), while the other half were personalized messages based on user's site usage (variant="personalization"). Asuming that a 95% confidence interval for statistical significance (p-value = 0.05 in T-test). Determine the conversion rate lift and it's statistical significance both overall and for each age group.

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Calculates the lift
def get_lift(a,b):
    a_mean = np.mean(a)
    b_mean = np.mean(b)

    lift = (b_mean - a_mean) / a_mean
    return str(round(lift,2)) 

# Calculates the lift and runs an independent t-test for every segment
def test_segment(segment):
  for subsegment in np.unique(marketing[segment].values):
    print(subsegment)
      
    # Limit marketing to email and subsegment      
    email = marketing[(marketing["marketing_channel"] == "Email") & (marketing[segment] == subsegment)]

    subscribers = email.groupby(["user_id", "variant"])["converted"].max()
    subscribers = pd.DataFrame(subscribers.unstack(level=1)) 
    control = pd.DataFrame(subscribers["control"].dropna())
    personalization = pd.DataFrame(subscribers["personalization"].dropna())
    control["control"]=control["control"].astype(int)
    personalization["personalization"]=personalization["personalization"].astype(int)
      
    print("Lift: ", get_lift(control, personalization))
    t_stat,p_value = stats.ttest_ind(control, personalization)
    print("p_value: " + str(p_value))

    if p_value < 0.05:
        print("The Lift is statistically significant for this segment\n")
    else:
        print("The Lift is not statistically signiciant for this segment\n")

# Load data and transform
marketing=pd.read_csv("marketing.csv", parse_dates=["date_served","date_subscribed","date_canceled"])
marketing['is_retained']=marketing["is_retained"].astype("bool")

C:\Users\markd\AppData\Local\Temp\ipykernel_13764\2253303890.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  marketing=pd.read_csv("marketing.csv", parse_dates=["date_served","date_subscribed","date_canceled"])
C:\Users\markd\AppData\Local\Temp\ipykernel_13764\2253303890.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  marketing=pd.read_csv("marketing.csv", parse_dates=["date_served","date_subscribed","date_canceled"])
C:\Users\markd\AppData\Local\Temp\ipykernel_13764\2253303890.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  marketing=pd.read_csv("marketing.c

In [64]:
# Determine Conversion Rates and lift
sub = email.groupby(["user_id","variant"])["converted"].max()
df_sub = pd.DataFrame(sub.unstack(level=1))
control = pd.DataFrame(df_sub["control"].dropna())
personalization = pd.DataFrame(df_sub["personalization"].dropna())

print("OVERALL CONVERSION RATE") 
print("Control Conversion rate: " + str(np.mean(control)))
print("Personalization Conversion rate: " + str(np.mean(personalization)))
print("Lift: " + get_lift(control,personalization))

# Convert to numeric and perform indepedent t-test
control["control"]=control["control"].astype(int)
personalization["personalization"]=personalization["personalization"].astype(int)
t_stat,p_value = stats.ttest_ind(personalization,control)
print("p-value : " + str(p_value))
if p_value < 0.05:
    print("The lift is statistically significant\n\n")
else:
    print("The lift is not statistically significant\n\n")

# Determine lift for each age group and run independent t-test
print("CONVERSION RATE LIFT BY AGE GROUP")
test_segment("age_group")

OVERALL CONVERSION RATE
Control Conversion rate: 0.2814814814814815
Personalization Conversion rate: 0.3908450704225352
Lift: 0.39
p-value : [0.00645149]
The lift is statistically significant


CONVERSION RATE LIFT BY AGE GROUP
0-18 years
Lift:  1.21
p_value: [0.00387245]
The Lift is statistically significant for this segment

19-24 years
Lift:  1.06
p_value: [0.00306238]
The Lift is statistically significant for this segment

24-30 years
Lift:  1.61
p_value: [0.00018743]
The Lift is statistically significant for this segment

30-36 years
Lift:  -1.0
p_value: [0.00232385]
The Lift is statistically significant for this segment

36-45 years
Lift:  -0.85
p_value: [0.01797569]
The Lift is statistically significant for this segment

45-55 years
Lift:  -0.72
p_value: [0.04306234]
The Lift is statistically significant for this segment

55+ years
Lift:  -1.0
p_value: [0.00163586]
The Lift is statistically significant for this segment



The personalized emails resulted in a 39% lift in conversions. This is statistically significant with 95% confidence. The lift was positive for customers younger than 30, while negaive for older customers. The lifts are statistically significant for all age groups. 